In [8]:
import arcpy

# -------------------------------------------------------------------
# User-defined parameters
# -------------------------------------------------------------------
parcels_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\TaxPlat_NoConservation\TaxPlat_NoConservation.shp"
sewer_mains_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\HonoluluSewerBuffer_v2\HonoluluSewerBuffer_v2.shp"
output_erase_fc = r"C:\Users\Sea Grant\Documents\GitHub\WW_Overlay_2024\Data\Outputs\Parcels_No_Overlap.shp"

# Field to store area percentages
original_area_field = "Orig_Area"
remaining_area_field = "Remn_Area"
percent_remaining_field = "Pct_Remn"

# -------------------------------------------------------------------
# 1. Calculate Original Parcel Area
# -------------------------------------------------------------------
print("Calculating original parcel areas...")
if original_area_field not in [f.name for f in arcpy.ListFields(parcels_fc)]:
    arcpy.management.AddField(parcels_fc, original_area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        in_features=parcels_fc,
        geometry_property=[[original_area_field, "AREA_GEODESIC"]],
        area_unit="SQUARE_METERS"
    )

# -------------------------------------------------------------------
# 2. Perform Erase to Remove Overlapping Areas
# -------------------------------------------------------------------
print("Removing overlapping areas...")
arcpy.analysis.Erase(
    in_features=parcels_fc,
    erase_features=sewer_mains_fc,
    out_feature_class=output_erase_fc
)

# -------------------------------------------------------------------
# 3. Calculate Remaining Parcel Area After Erase
# -------------------------------------------------------------------
print("Calculating remaining parcel areas...")
if remaining_area_field not in [f.name for f in arcpy.ListFields(output_erase_fc)]:
    arcpy.management.AddField(output_erase_fc, remaining_area_field, "DOUBLE")
    arcpy.management.CalculateGeometryAttributes(
        in_features=output_erase_fc,
        geometry_property=[[remaining_area_field, "AREA_GEODESIC"]],
        area_unit="SQUARE_METERS"
    )

# -------------------------------------------------------------------
# 4. Add Field for Percentage of Area Left
# -------------------------------------------------------------------
print("Calculating percentage of area remaining...")
if percent_remaining_field not in [f.name for f in arcpy.ListFields(output_erase_fc)]:
    arcpy.management.AddField(output_erase_fc, percent_remaining_field, "DOUBLE")

# -------------------------------------------------------------------
# 5. Compute Percentage of Remaining Area
# -------------------------------------------------------------------
parcel_dict = {}
with arcpy.da.SearchCursor(parcels_fc, ["OBJECTID", original_area_field]) as orig_cursor:
    for row in orig_cursor:
        parcel_dict[row[0]] = row[1]  # Store original area by OBJECTID

with arcpy.da.UpdateCursor(output_erase_fc, ["OBJECTID", remaining_area_field, percent_remaining_field]) as update_cursor:
    for row in update_cursor:
        object_id = row[0]
        original_area = parcel_dict.get(object_id, 0.0)
        remaining_area = row[1]

        if original_area > 0:
            percent_remaining = (remaining_area / original_area) * 100
        else:
            percent_remaining = 0.0

        row[2] = percent_remaining
        update_cursor.updateRow(row)

print("Percentage of remaining area calculation complete.")


Calculating original parcel areas...
Removing overlapping areas...
Calculating remaining parcel areas...
Calculating percentage of area remaining...
Percentage of remaining area calculation complete.
